In [ ]:
from DeepFMKit.experiments import Experiment

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial

print("DeepFMKit modules loaded successfully.")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Pandas version: {pd.__version__}")

In [ ]:
def amplitude_random_offset_generator(nominal_amplitude: float, relative_noise_std: float) -> float:
    """
    Generates a random amplitude offset whose standard deviation scales
    with the `nominal_amplitude`. This simulates proportional noise.

    Parameters
    ----------
    nominal_amplitude : float
        The nominal signal amplitude, which dictates the scale of the random offset.

    Returns
    -------
    float
        A single random offset value.
    """
    noise_std = nominal_amplitude * relative_noise_std
    return np.random.normal(loc=0.0, scale=noise_std)

In [ ]:
from DeepFMKit.factories import VairableAmplitudeOffset
"""
VairableAmplitudeOffset::__call__():
    m_main = params['m_main']
    nominal_amplitude = params['nominal_amplitude']
    amplitude_offset = params['amplitude_offset']
"""

my_factory_instance = VairableAmplitudeOffset(
    opd_main=0.1
)

# I create an Experiment object.
experiment = Experiment(description="Example using VairableAmplitudeOffset factory")
experiment.set_config_factory(my_factory_instance)

# I define my sweep axis: the nominal signal amplitude.
# I will vary it from 0.5 to 2.0 in 10 steps.
nominal_amplitudes = np.linspace(0.5, 2.0, 10)
experiment.add_axis('nominal_amplitude', nominal_amplitudes)

# I define static parameters for all trials.
experiment.set_static({
    'm_main': 5.0,                  # Keep modulation depth constant
    'n_fit_buffers_per_trial': 20,  # 20 cycles per fit buffer
    'f_samp': 200000                # Sampling frequency
})

# I add my stochastic variable: a random offset to the signal amplitude.
# It 'depends_on' the 'nominal_amplitude' axis.

gen_fun = partial(amplitude_random_offset_generator, relative_noise_std=0.05)

experiment.add_stochastic_variable(
    name='amplitude_offset',
    generator_func=gen_fun,
    depends_on='nominal_amplitude' # This links the noise scale to the nominal amplitude
)

# I specify the number of Monte Carlo trials for each point on the axis.
experiment.n_trials = 100 # 50 trials per nominal_amplitude point for good statistics

# I define the analysis to be performed on each simulated data set.
# I will use the NLS fitter and specify which columns to collect from its results.
experiment.add_analysis(
    name='NLS_Fit',
    fitter_method='nls',
    result_cols=['amp', 'm', 'phi', 'psi', 'ssq'], # Parameters I want to collect
    fitter_kwargs={
        'n': 20,       # 20 modulation cycles per fit buffer
        'ndata': 10,   # Use this many harmonics for NLS
    }
)

# --- 6. Run the Experiment ---
print(f"Starting experiment: '{experiment.description}'...")
experiment.results = experiment.run()
print("Experiment completed.")

In [ ]:
# --- 7. Analyze and Plot Results ---
results = experiment.results
nominal_amp_axis = results['axes']['nominal_amplitude']

# --- Plot 1: Mean Fitted Amplitude vs. Nominal Amplitude ---
fig1, ax1 = plt.subplots(figsize=(10, 6))
mean_fitted_amp = results['NLS_Fit']['amp']['mean']
std_fitted_amp = results['NLS_Fit']['amp']['std']

ax1.errorbar(nominal_amp_axis, mean_fitted_amp, yerr=std_fitted_amp, fmt='o-', capsize=5, label='Mean Fitted Amplitude $\\pm 1\\sigma$')
ax1.plot(nominal_amp_axis, nominal_amp_axis, 'k--', label='Expected (Nominal Amplitude)') # Ideal line: fitted amp == nominal amp

ax1.set_xlabel('Nominal Signal Amplitude (a.u.)')
ax1.set_ylabel('Fitted Amplitude (a.u.)')
ax1.set_title('Mean Fitted Signal Amplitude vs. Nominal Amplitude')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- Plot 2: Relative Standard Deviation of Fitted Amplitude ---
fig2, ax2 = plt.subplots(figsize=(10, 6))
# Calculate the expected standard deviation from the generator function.
# This assumes the relative_noise_std (0.02) is the standard deviation on the OFFSET,
# so the standard deviation of the fitted amplitude should reflect this.
expected_std_amplitude = nominal_amp_axis * 0.05 # 2% of nominal amplitude
ax2.plot(nominal_amp_axis, std_fitted_amp / mean_fitted_amp, 'o-', label='Relative Std Dev of Fitted Amplitude')
ax2.plot(nominal_amp_axis, expected_std_amplitude / nominal_amp_axis, 'k--', label='Expected Relative Std Dev (2%)')

ax2.set_xlabel('Nominal Signal Amplitude (a.u.)')
ax2.set_ylabel('Relative Standard Deviation ($\\sigma_{amp} / \\langle A \\rangle$)')
ax2.set_title('Relative Standard Deviation of Fitted Signal Amplitude')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
from DeepFMKit.factories import VairableSNR
"""
VairableSNR::__call__():
    m_main = params['m_main']
    snr_dB = params['snr_dB']
    f_samp = params['f_samp']
"""

my_factory_instance = VairableSNR(
    opd_main=0.1
)

# I create an Experiment object.
experiment = Experiment(description="Example using VairableSNR factory")
experiment.set_config_factory(my_factory_instance)

# I define my sweep axis
nominal_amplitudes = np.linspace(0.0, 100.0, 100)
experiment.add_axis('snr_dB', nominal_amplitudes)

# I define static parameters for all trials.
experiment.set_static({
    'm_main': 5.0,                  # Keep modulation depth constant
    'f_samp': 200000                # Sampling frequency
})

experiment.n_trials = 50

# I define the analysis to be performed on each simulated data set.
# I will use the NLS fitter and specify which columns to collect from its results.
experiment.add_analysis(
    name='NLS_Fit',
    fitter_method='nls',
    result_cols=['amp', 'm', 'phi', 'psi', 'ssq'], # Parameters I want to collect
    fitter_kwargs={
        'n': 1,       # 20 modulation cycles per fit buffer
        'ndata': 10,   # Use this many harmonics for NLS
    }
)

# --- 6. Run the Experiment ---
print(f"Starting experiment: '{experiment.description}'...")
experiment.results = experiment.run()
print("Experiment completed.")

In [ ]:
# --- 7. Analyze and Plot Results ---
results = experiment.results
snr_dB_axis = results['axes']['snr_dB']

# --- Plot 1: Mean Fitted Amplitude vs. Nominal Amplitude ---
fig1, ax1 = plt.subplots(figsize=(10, 6))
m_mean = results['NLS_Fit']['m']['mean']
m_std = results['NLS_Fit']['m']['std']

ax1.errorbar(snr_dB_axis, m_mean, yerr=m_std, fmt='o-', capsize=5, label='Mean Fitted Amplitude $\\pm 1\\sigma$')
# ax1.plot(snr_dB_axis, snr_dB_axis, 'k--', label='Expected (Nominal Amplitude)') # Ideal line: fitted amp == nominal amp

ax1.set_xlabel('Nominal Signal Amplitude (a.u.)')
ax1.set_ylabel('Fitted Amplitude (a.u.)')
ax1.set_title('Mean Fitted Signal Amplitude vs. Nominal Amplitude')
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# --- Plot 2: Relative Standard Deviation of Fitted Amplitude ---
fig2, ax2 = plt.subplots(figsize=(10, 6))
ax2.plot(snr_dB_axis, m_std, 'o-', label='Relative Std Dev of Fitted Amplitude')
ax2.plot(snr_dB_axis, 1.25e-2*np.sqrt(8)/(10**(snr_dB_axis/20)), 'o-', label='Relative Std Dev of Fitted Amplitude')


ax2.set_xlabel('Nominal Signal Amplitude (a.u.)')
ax2.set_ylabel('Relative Standard Deviation ($\\sigma_{amp} / \\langle A \\rangle$)')
ax2.set_title('Relative Standard Deviation of Fitted Signal Amplitude')
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()